In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
import pandas as pd
from PIL import Image
from scipy import ndimage


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('./train.csv') 
df1 = pd.read_csv('./test.csv') 

In [3]:
df

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3063,pixel_3064,pixel_3065,pixel_3066,pixel_3067,pixel_3068,pixel_3069,pixel_3070,pixel_3071,label
0,59,43,50,68,98,119,139,145,149,149,...,58,65,59,46,57,104,140,84,72,6
1,154,126,105,102,125,155,172,180,142,111,...,42,67,101,122,133,136,139,142,144,9
2,255,253,253,253,253,253,253,253,253,253,...,83,80,69,66,72,79,83,83,84,9
3,28,37,38,42,44,40,40,24,32,43,...,39,59,42,44,48,38,28,37,46,4
4,170,168,177,183,181,177,181,184,189,189,...,88,85,82,83,79,78,82,78,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,145,151,156,155,153,153,157,161,161,161,...,68,64,61,63,59,57,56,53,54,2
49996,255,254,255,254,254,254,254,253,254,254,...,61,64,68,64,76,82,79,81,76,6
49997,35,40,42,39,43,45,48,51,53,56,...,94,104,103,96,93,87,77,66,50,9
49998,189,186,185,184,182,181,180,180,181,180,...,166,171,171,172,169,165,169,171,171,1


In [4]:
Gooddate = df.to_numpy()
NotBad= df1.to_numpy()

In [5]:
Gooddate

array([[ 59,  43,  50, ...,  84,  72,   6],
       [154, 126, 105, ..., 142, 144,   9],
       [255, 253, 253, ...,  83,  84,   9],
       ...,
       [ 35,  40,  42, ...,  66,  50,   9],
       [189, 186, 185, ..., 171, 171,   1],
       [229, 236, 234, ..., 162, 161,   1]], dtype=int64)

In [ ]:
train_set_x1=Gooddate[:,:-1].transpose()
train_set_y1=Gooddate[:,-1].reshape(1,-1)
test_set_x1 = Gooddate[20000:,:-1].transpose()
test_set_y1 = Gooddate[20000:,-1].reshape(1,-1)


In [ ]:
train_set_y1

In [ ]:
train_set_y1[train_set_y1 != 1] = 0

In [ ]:
train_set_y1

In [ ]:
test_set_x1.shape


In [ ]:

m_train = train_set_x1.shape[0]
m_test = test_set_x1.shape[0]
num_px = train_set_x1.shape[1]



print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x1.shape))
print ("train_set_y shape: " + str(train_set_y1.shape))
print ("test_set_x shape: " + str(test_set_x1.shape))
print ("test_set_y shape: " + str(test_set_y1.shape))

In [ ]:

train_set_x_flatten = train_set_x1.reshape(train_set_x1.shape[0],-1).T
test_set_x_flatten = test_set_x1.reshape(test_set_x1.shape[0],-1).T


print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y1.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y1.shape))

In [ ]:
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

In [ ]:
train_set_x1

In [ ]:
test_set_y1

In [ ]:
train_set_y1.shape

In [6]:
def sigmoid(z):
    s=1/(1+(np.exp(-z)))
    return s


In [39]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid([0, 2]) = [0.5        0.88079708]


In [44]:

def initialize_with_zeros(dim):

    w = np.zeros([dim,1])
    b = 0.0

    return w, b

In [46]:
dim = 2
w, b = initialize_with_zeros(dim)

assert type(b) == float
print ("w = " + str(w))
print ("b = " + str(b))

w = [[0.]
 [0.]]
b = 0.0


In [49]:
def propagate (w,b,X,Y):
    m= X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)
    cost= -1/m * (np.dot(Y,np.log(A).T) + np.dot((1-Y),np.log(1 - A).T))
    dw = 1 / m *(np.dot(X,(A - Y).T))
    db = 1 / m *(np.sum(A - Y))
    cost = np.squeeze(np.array(cost))
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [50]:
w =  np.array([[1.], [2]])
b = 1.5
X = np.array([[1., -2., -1.], [3., 0.5, -3.2]])
Y = np.array([[1, 1, 0]])
grads, cost = propagate(w, b, X, Y)

assert type(grads["dw"]) == np.ndarray
assert grads["dw"].shape == (2, 1)
assert type(grads["db"]) == np.float64


print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))



dw = [[ 0.25071532]
 [-0.06604096]]
db = -0.1250040450043965
cost = 0.15900537707692405


In [ ]:

def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):

    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):

        grads, cost = propagate(w,b,X,Y)
        

        dw = grads["dw"]
        db = grads["db"]
        

        w = w - learning_rate*dw
        b = b - learning_rate*db

        if i % 100 == 0:
            costs.append(cost)

            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [52]:
params, grads, costs = optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print("Costs = " + str(costs))



w = [[0.80956046]
 [2.0508202 ]]
b = 1.5948713189708588
dw = [[ 0.17860505]
 [-0.04840656]]
db = -0.08888460336847771
Costs = [array(0.15900538)]


In [15]:

def predict(w, b, X):

    m =  X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    

    A = sigmoid(np.dot(w.T,X) + b)

    
    for i in range(A.shape[1]):
        
        if A[0, i] <=0.5:
            Y_prediction[0,i] =0
        else:
            Y_prediction[0,i] =1

    return Y_prediction

In [16]:

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):

    w, b = initialize_with_zeros(X_train.shape[0])
    params, grads, costs =  optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    w = params["w"]
    b = params["b"]
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    

    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [61]:
logistic_regression_model = model(train_set_x, train_set_y1, test_set_x, test_set_y1, num_iterations=2000, learning_rate=0.005, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.310266
Cost after iteration 200: 0.299360
Cost after iteration 300: 0.292099
Cost after iteration 400: 0.286807
Cost after iteration 500: 0.282798
Cost after iteration 600: 0.279663
Cost after iteration 700: 0.277140
Cost after iteration 800: 0.275054
Cost after iteration 900: 0.273290
Cost after iteration 1000: 0.271765
Cost after iteration 1100: 0.270426
Cost after iteration 1200: 0.269231
Cost after iteration 1300: 0.268152
Cost after iteration 1400: 0.267169
Cost after iteration 1500: 0.266265
Cost after iteration 1600: 0.265429
Cost after iteration 1700: 0.264650
Cost after iteration 1800: 0.263921
Cost after iteration 1900: 0.263237
train accuracy: 90.394 %
test accuracy: 90.38 %


In [33]:
train_set_x.shape

(3072, 50000)

In [1]:
# change this to the name of your image file
my_image = "my_image.jpg"   

# We preprocess the image to fit your algorithm.
fname = "images/" + my_image
image = np.array(Image.open(fname).resize((num_px, num_px)))
plt.imshow(image)
image = image / 255.
image = image.reshape((1, num_px * num_px * 3)).T
my_predicted_image = predict(logistic_regression_model["w"], logistic_regression_model["b"], image)

print("y = " + str(np.squeeze(my_predicted_image)) + ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")

NameError: name 'np' is not defined

Bibliography:
- http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
- https://stats.stackexchange.com/questions/211436/why-do-we-normalize-images-by-subtracting-the-datasets-image-mean-and-not-the-c